In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/handover_TA/Thesis

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/103ZfdM-NIuqvP3_1L00yO_8eGukJPJWe/handover_TA/Thesis


In [2]:
!pip install tensorflow-hub

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
np.random.seed(20242023)

import seaborn as sns
from sklearn.metrics import confusion_matrix, accuracy_score

#NLP Libraries
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

#Deep Learning Libraries
import tensorflow as tf
tf.random.set_seed(20242023)
from tensorflow.keras.layers import Embedding
import tensorflow_hub as hub
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import Bidirectional

#Vocabulary Size
vocab_size=5000

In [4]:
from keras import backend as K
import numpy as np
import tensorflow as tf


def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
# !mkdir prediction_BiLSTM
# %cd prediction_BiLSTM
# !mkdir result_classification_BiLSTM
# %cd ..

In [5]:
import math
from sklearn.metrics import f1_score, recall_score, precision_score
from sklearn.metrics import precision_recall_fscore_support

def display_score_format(fl,pl,rl):
    print(f"Nilai maksimal dari semua run:")
    print("P =", max(pl))
    print("R =", max(rl))
    print("F1 =", max(fl))

    print(f"Nilai minimal dari semua run:")
    print("P =", min(pl))
    print("R =", min(rl))
    print("F1 =", min(fl))

    print(f"Nilai rata-rata dari semua run:")
    print("P =", sum(pl)/len(pl))
    print("R =", sum(rl)/len(rl))
    print("F1 =", sum(fl)/len(fl))

In [6]:
def get_score_f1_bencana_tuned(bencana,list_kolom, num_run):
    list_kolom.append('id')
    list_kolom.append('label')
    df_train = pd.read_csv(f'./dataset/{bencana}/{bencana}_train.csv')[list_kolom]#.sample(frac = 1)
    df_dev = pd.read_csv(f'./dataset/{bencana}/{bencana}_dev.csv')[list_kolom]
    df_test = pd.read_csv(f'./dataset/{bencana}/{bencana}_test.csv')[list_kolom]
    df_train[list_kolom[0]] = df_train[list_kolom[0]].str.lower()
    df_dev[list_kolom[0]] = df_dev[list_kolom[0]].str.lower()
    df_test[list_kolom[0]] = df_test[list_kolom[0]].str.lower()


    corpus_train = list(df_train[list_kolom[0]])
    corpus_test = list(df_test[list_kolom[0]])
    corpus_dev = list(df_dev[list_kolom[0]])

    voc_size = 50000

    encoding_train = [one_hot(words,voc_size) for words in corpus_train]
    encoding_test = [one_hot(words,voc_size) for words in corpus_test]
    encoding_dev = [one_hot(words,voc_size) for words in corpus_dev]

    emb_doc_train = pad_sequences(encoding_train,padding='post',maxlen=256)
    emb_doc_test = pad_sequences(encoding_test,padding='post',maxlen=256)
    emb_doc_dev = pad_sequences(encoding_dev,padding='post',maxlen=256)

    # embedding = "https://tfhub.dev/google/nnlm-en-dim50/2"
    # hub_layer = hub.KerasLayer(embedding, input_shape=[],
    #                           dtype=tf.string, trainable=True)

    model = Sequential()
    model.add(Embedding(voc_size,160,input_length=256)) # making embedding layer
    model.add(Bidirectional(LSTM(256,  return_sequences=True)))  # one LSTM Layer with 100 neurons
    model.add(Bidirectional(LSTM(128)))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=[f1_m, 'acc'] )


    LABEL2INDEX = { 'N':0,  'Y':1}

    y_train = list(df_train['label'].apply(lambda lab: LABEL2INDEX[lab]))
    y_test = list(df_test['label'].apply(lambda lab: LABEL2INDEX[lab]))
    y_dev = list(df_dev['label'].apply(lambda lab: LABEL2INDEX[lab]))

    x_final_train= np.array(emb_doc_train)
    y_final_train= np.array(y_train)

    x_final_test= np.array(emb_doc_test)
    y_final_test= np.array(y_test)

    x_final_dev= np.array(emb_doc_dev)
    y_final_dev= np.array(y_dev)

    model.fit(x_final_train,y_final_train,validation_data=(x_final_dev,y_final_dev),epochs=10,batch_size=64)

    predict_x_final_test = model.predict(x_final_test)

    y_pred = []
    threshold = 0.5
    for i in range(len(predict_x_final_test)):
        if predict_x_final_test[i][0]>=threshold:
            y_pred.append(1)
        else:
            y_pred.append(0)

    predict_x_final_train=model.predict(x_final_train)
    y_pred_train = []

    for i in range(len(predict_x_final_train)):
        if predict_x_final_train[i][0]>=threshold:
            y_pred_train.append(1)
        else:
            y_pred_train.append(0)
    print("Pada data training")
    print(confusion_matrix(y_final_train,y_pred_train))
    p,r, f,s = precision_recall_fscore_support(y_final_train,y_pred_train, average='binary', pos_label=1)
    print(f"Nilai Precision = {p}")
    print(f"Nilai Recall = {r}")
    print(f"Nilai F1 = {f}")

    print("Pada data test")
    count = 1

    print(confusion_matrix(y_final_test,y_pred))
    df = df_test[['id']]
    df['label'] = y_pred
    df[['id','label']].to_csv(f'./prediction_BiLSTM/result_classification_BiLSTM/{bencana}_{list_kolom[0]}_{num_run}.csv', index=False)
    p,r, f,s = precision_recall_fscore_support(y_final_test,y_pred, average='binary', pos_label=1)
    print(f"Nilai Precision = {p}")
    print(f"Nilai Recall = {r}")
    print(f"Nilai F1 = {f}")
    return p,  r, f

In [7]:
list_kolom = ['title_ori', 'text_ori', 'tc_ori', 'filter_word_tc',
       'filter_word_tc_window', 'textrank', 'sum_indobart', 'sum_liputan6',
       'sum_pegasus', 'trunc_first_short', 'trunc_last_short',
       'trunc_first_long', 'trunc_last_long', 'trunc_combination']
def get_f1_each_representation(bencana, num_run):
  for fitur in list_kolom:
    print(f'Running for representation {fitur}')
    get_score_f1_bencana_tuned(bencana,[fitur], num_run)
    print('selesai')

In [8]:
for i in range(1):
  get_f1_each_representation('angin_topan', i+1)

Running for representation title_ori
Epoch 1/10
15/15 [==============================] - 25s 563ms/step - loss: 0.5116 - f1_m: 0.0213 - acc: 0.7824 - val_loss: 0.6628 - val_f1_m: 0.0000e+00 - val_acc: 0.6800
Epoch 2/10
15/15 [==============================] - 5s 336ms/step - loss: 0.4020 - f1_m: 0.0242 - acc: 0.8275 - val_loss: 0.5071 - val_f1_m: 0.5741 - val_acc: 0.8100
Epoch 3/10
15/15 [==============================] - 3s 195ms/step - loss: 0.1496 - f1_m: 0.7829 - acc: 0.9505 - val_loss: 0.4712 - val_f1_m: 0.7947 - val_acc: 0.8700
Epoch 4/10
15/15 [==============================] - 2s 150ms/step - loss: 0.0627 - f1_m: 0.9220 - acc: 0.9758 - val_loss: 0.5536 - val_f1_m: 0.7875 - val_acc: 0.8700
Epoch 5/10
15/15 [==============================] - 4s 271ms/step - loss: 0.0342 - f1_m: 0.9758 - acc: 0.9912 - val_loss: 0.6877 - val_f1_m: 0.7413 - val_acc: 0.8300
Epoch 6/10
15/15 [==============================] - 2s 121ms/step - loss: 0.0160 - f1_m: 0.9793 - acc: 0.9945 - val_loss: 0.8174

<ipython-input-6-957c9328e0e5>:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred


Nilai Precision = 0.813953488372093
Nilai Recall = 0.7608695652173914
Nilai F1 = 0.7865168539325844
selesai
Running for representation text_ori
Epoch 1/10
15/15 [==============================] - 12s 335ms/step - loss: 0.5445 - f1_m: 0.0194 - acc: 0.7956 - val_loss: 0.6369 - val_f1_m: 0.0000e+00 - val_acc: 0.6800
Epoch 2/10
15/15 [==============================] - 4s 248ms/step - loss: 0.2716 - f1_m: 0.4528 - acc: 0.8890 - val_loss: 0.2736 - val_f1_m: 0.8128 - val_acc: 0.8800
Epoch 3/10
15/15 [==============================] - 4s 230ms/step - loss: 0.0840 - f1_m: 0.8715 - acc: 0.9780 - val_loss: 0.2813 - val_f1_m: 0.8054 - val_acc: 0.8800
Epoch 4/10
15/15 [==============================] - 3s 207ms/step - loss: 0.0334 - f1_m: 0.9831 - acc: 0.9945 - val_loss: 0.5590 - val_f1_m: 0.7446 - val_acc: 0.8600
Epoch 5/10
15/15 [==============================] - 3s 209ms/step - loss: 0.0169 - f1_m: 0.9911 - acc: 0.9967 - val_loss: 0.3988 - val_f1_m: 0.7990 - val_acc: 0.8800
Epoch 6/10
15/15 [===

<ipython-input-6-957c9328e0e5>:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred


Nilai Precision = 0.9142857142857143
Nilai Recall = 0.6956521739130435
Nilai F1 = 0.7901234567901234
selesai
Running for representation tc_ori
Epoch 1/10
15/15 [==============================] - 13s 368ms/step - loss: 0.5504 - f1_m: 0.0216 - acc: 0.8110 - val_loss: 0.6136 - val_f1_m: 0.0000e+00 - val_acc: 0.6800
Epoch 2/10
15/15 [==============================] - 3s 220ms/step - loss: 0.2708 - f1_m: 0.5254 - acc: 0.8967 - val_loss: 0.2601 - val_f1_m: 0.8454 - val_acc: 0.9000
Epoch 3/10
15/15 [==============================] - 4s 238ms/step - loss: 0.0822 - f1_m: 0.8677 - acc: 0.9791 - val_loss: 0.4163 - val_f1_m: 0.7639 - val_acc: 0.8700
Epoch 4/10
15/15 [==============================] - 3s 212ms/step - loss: 0.0257 - f1_m: 0.9846 - acc: 0.9956 - val_loss: 0.4443 - val_f1_m: 0.7048 - val_acc: 0.8500
Epoch 5/10
15/15 [==============================] - 3s 189ms/step - loss: 0.0162 - f1_m: 0.9940 - acc: 0.9978 - val_loss: 0.5169 - val_f1_m: 0.7358 - val_acc: 0.8600
Epoch 6/10
15/15 [====

...

Nilai Precision = 0.85
Nilai Recall = 0.7391304347826086
Nilai F1 = 0.7906976744186046
selesai
Running for representation trunc_last_long
Epoch 1/10
15/15 [==============================] - 12s 327ms/step - loss: 0.5221 - f1_m: 0.0145 - acc: 0.7989 - val_loss: 0.6280 - val_f1_m: 0.0000e+00 - val_acc: 0.6800
Epoch 2/10
15/15 [==============================] - 4s 256ms/step - loss: 0.3700 - f1_m: 0.2244 - acc: 0.8538 - val_loss: 0.4215 - val_f1_m: 0.5572 - val_acc: 0.7800
Epoch 3/10
15/15 [==============================] - 3s 213ms/step - loss: 0.1262 - f1_m: 0.8205 - acc: 0.9560 - val_loss: 0.5879 - val_f1_m: 0.5722 - val_acc: 0.7900
Epoch 4/10
15/15 [==============================] - 3s 182ms/step - loss: 0.0404 - f1_m: 0.9807 - acc: 0.9934 - val_loss: 0.6061 - val_f1_m: 0.6310 - val_acc: 0.8100
Epoch 5/10
15/15 [==============================] - 3s 185ms/step - loss: 0.0117 - f1_m: 0.9977 - acc: 0.9989 - val_loss: 0.8893 - val_f1_m: 0.6310 - val_acc: 0.8100
Epoch 6/10
15/15 [=========

<ipython-input-6-957c9328e0e5>:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred


Nilai Precision = 0.875
Nilai Recall = 0.30434782608695654
Nilai F1 = 0.4516129032258065
selesai
Running for representation trunc_combination
Epoch 1/10
15/15 [==============================] - 12s 344ms/step - loss: 0.4885 - f1_m: 0.0000e+00 - acc: 0.8242 - val_loss: 0.4234 - val_f1_m: 0.5045 - val_acc: 0.7900
Epoch 2/10
15/15 [==============================] - 3s 237ms/step - loss: 0.1687 - f1_m: 0.7575 - acc: 0.9385 - val_loss: 0.2585 - val_f1_m: 0.8514 - val_acc: 0.9100
Epoch 3/10
15/15 [==============================] - 4s 235ms/step - loss: 0.0430 - f1_m: 0.9057 - acc: 0.9890 - val_loss: 0.7128 - val_f1_m: 0.6015 - val_acc: 0.8100
Epoch 4/10
15/15 [==============================] - 3s 201ms/step - loss: 0.0140 - f1_m: 0.9898 - acc: 0.9956 - val_loss: 0.4146 - val_f1_m: 0.8348 - val_acc: 0.9000
Epoch 5/10
15/15 [==============================] - 3s 184ms/step - loss: 0.0094 - f1_m: 0.9954 - acc: 0.9978 - val_loss: 0.4329 - val_f1_m: 0.8167 - val_acc: 0.8900
Epoch 6/10
15/15 [=====

<ipython-input-6-957c9328e0e5>:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred


Nilai Precision = 0.8928571428571429
Nilai Recall = 0.5434782608695652
Nilai F1 = 0.6756756756756757
selesai


In [9]:
for i in range(1):
  get_f1_each_representation('banjir', i+1)

Running for representation title_ori
Epoch 1/10
15/15 [==============================] - 11s 279ms/step - loss: 0.5781 - f1_m: 0.0211 - acc: 0.7222 - val_loss: 0.7840 - val_f1_m: 0.0000e+00 - val_acc: 0.5833
Epoch 2/10
15/15 [==============================] - 3s 168ms/step - loss: 0.5094 - f1_m: 0.0000e+00 - acc: 0.7639 - val_loss: 0.5916 - val_f1_m: 0.1323 - val_acc: 0.6111
Epoch 3/10
15/15 [==============================] - 2s 164ms/step - loss: 0.1725 - f1_m: 0.8719 - acc: 0.9434 - val_loss: 0.6750 - val_f1_m: 0.6889 - val_acc: 0.7778
Epoch 4/10
15/15 [==============================] - 2s 144ms/step - loss: 0.0524 - f1_m: 0.9694 - acc: 0.9861 - val_loss: 0.7879 - val_f1_m: 0.7029 - val_acc: 0.7870
Epoch 5/10
15/15 [==============================] - 2s 133ms/step - loss: 0.0040 - f1_m: 1.0000 - acc: 1.0000 - val_loss: 1.1654 - val_f1_m: 0.6742 - val_acc: 0.7685
Epoch 6/10
15/15 [==============================] - 2s 131ms/step - loss: 0.0011 - f1_m: 1.0000 - acc: 1.0000 - val_loss: 1.

<ipython-input-6-957c9328e0e5>:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred


Nilai Precision = 0.7428571428571429
Nilai Recall = 0.48148148148148145
Nilai F1 = 0.5842696629213483
selesai
Running for representation text_ori
Epoch 1/10
15/15 [==============================] - 11s 319ms/step - loss: 0.5861 - f1_m: 0.0215 - acc: 0.7329 - val_loss: 0.6738 - val_f1_m: 0.0000e+00 - val_acc: 0.5833
Epoch 2/10
15/15 [==============================] - 4s 253ms/step - loss: 0.3367 - f1_m: 0.5999 - acc: 0.8654 - val_loss: 0.4425 - val_f1_m: 0.7735 - val_acc: 0.8426
Epoch 3/10
15/15 [==============================] - 3s 187ms/step - loss: 0.0912 - f1_m: 0.9413 - acc: 0.9712 - val_loss: 0.6933 - val_f1_m: 0.7321 - val_acc: 0.8148
Epoch 4/10
15/15 [==============================] - 3s 175ms/step - loss: 0.0286 - f1_m: 0.9887 - acc: 0.9957 - val_loss: 1.0097 - val_f1_m: 0.5726 - val_acc: 0.7315
Epoch 5/10
15/15 [==============================] - 3s 186ms/step - loss: 0.0123 - f1_m: 0.9930 - acc: 0.9968 - val_loss: 1.1301 - val_f1_m: 0.6827 - val_acc: 0.7870
Epoch 6/10
15/15 [=

<ipython-input-6-957c9328e0e5>:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred


Nilai Precision = 0.7
Nilai Recall = 0.3888888888888889
Nilai F1 = 0.5
selesai
Running for representation tc_ori
Epoch 1/10
15/15 [==============================] - 12s 361ms/step - loss: 0.5861 - f1_m: 0.0137 - acc: 0.7447 - val_loss: 0.7372 - val_f1_m: 0.0000e+00 - val_acc: 0.5833
Epoch 2/10
15/15 [==============================] - 3s 196ms/step - loss: 0.3405 - f1_m: 0.5727 - acc: 0.8600 - val_loss: 0.3684 - val_f1_m: 0.8592 - val_acc: 0.8889
Epoch 3/10
15/15 [==============================] - 3s 180ms/step - loss: 0.0824 - f1_m: 0.9477 - acc: 0.9733 - val_loss: 0.9509 - val_f1_m: 0.7321 - val_acc: 0.8148
Epoch 4/10
15/15 [==============================] - 3s 185ms/step - loss: 0.0126 - f1_m: 0.9932 - acc: 0.9968 - val_loss: 0.9602 - val_f1_m: 0.7203 - val_acc: 0.7963
Epoch 5/10
15/15 [==============================] - 4s 241ms/step - loss: 0.0613 - f1_m: 0.9597 - acc: 0.9840 - val_loss: 0.9637 - val_f1_m: 0.6410 - val_acc: 0.7685
Epoch 6/10
15/15 [==============================] - 

...

Nilai Precision = 0.6140350877192983
Nilai Recall = 0.6481481481481481
Nilai F1 = 0.6306306306306307
selesai
Running for representation trunc_last_long
Epoch 1/10
15/15 [==============================] - 12s 298ms/step - loss: 0.6007 - f1_m: 0.0226 - acc: 0.7361 - val_loss: 0.7461 - val_f1_m: 0.0000e+00 - val_acc: 0.5833
Epoch 2/10
15/15 [==============================] - 4s 267ms/step - loss: 0.4377 - f1_m: 0.2714 - acc: 0.8109 - val_loss: 0.4692 - val_f1_m: 0.7457 - val_acc: 0.7963
Epoch 3/10
15/15 [==============================] - 3s 205ms/step - loss: 0.1658 - f1_m: 0.8845 - acc: 0.9348 - val_loss: 0.7869 - val_f1_m: 0.6786 - val_acc: 0.7963
Epoch 4/10
15/15 [==============================] - 3s 210ms/step - loss: 0.0393 - f1_m: 0.9717 - acc: 0.9872 - val_loss: 0.7969 - val_f1_m: 0.6591 - val_acc: 0.7315
Epoch 5/10
15/15 [==============================] - 3s 199ms/step - loss: 0.0138 - f1_m: 0.9953 - acc: 0.9979 - val_loss: 0.8910 - val_f1_m: 0.6922 - val_acc: 0.7870
Epoch 6/10
15

<ipython-input-6-957c9328e0e5>:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred


Nilai Precision = 0.8125
Nilai Recall = 0.48148148148148145
Nilai F1 = 0.6046511627906976
selesai
Running for representation trunc_combination
Epoch 1/10
15/15 [==============================] - 12s 354ms/step - loss: 0.5915 - f1_m: 0.0086 - acc: 0.7489 - val_loss: 0.7370 - val_f1_m: 0.0000e+00 - val_acc: 0.5833
Epoch 2/10
15/15 [==============================] - 3s 173ms/step - loss: 0.7592 - f1_m: 0.1570 - acc: 0.6432 - val_loss: 0.8573 - val_f1_m: 0.0000e+00 - val_acc: 0.5833
Epoch 3/10
15/15 [==============================] - 2s 167ms/step - loss: 0.3472 - f1_m: 0.3933 - acc: 0.8291 - val_loss: 0.6409 - val_f1_m: 0.7132 - val_acc: 0.6759
Epoch 4/10
15/15 [==============================] - 3s 168ms/step - loss: 0.1601 - f1_m: 0.8922 - acc: 0.9423 - val_loss: 0.5485 - val_f1_m: 0.8184 - val_acc: 0.8519
Epoch 5/10
15/15 [==============================] - 3s 177ms/step - loss: 0.0905 - f1_m: 0.9526 - acc: 0.9786 - val_loss: 0.9111 - val_f1_m: 0.7065 - val_acc: 0.7870
Epoch 6/10
15/15 [

<ipython-input-6-957c9328e0e5>:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred


Nilai Precision = 0.7441860465116279
Nilai Recall = 0.5925925925925926
Nilai F1 = 0.6597938144329897
selesai


In [ ]:
for i in range(1):
  get_f1_each_representation('erupsi', i+1)

In [ ]:
for i in range(1):
  get_f1_each_representation('gempa', i+1)

In [ ]:
for i in range(1):
  get_f1_each_representation('karhutla', i+1)

In [ ]:
for i in range(1):
  get_f1_each_representation('kekeringan', i+1)

In [10]:
for i in range(1):
  get_f1_each_representation('longsor', i+1)

Running for representation title_ori
Epoch 1/10
15/15 [==============================] - 12s 343ms/step - loss: 0.5845 - f1_m: 0.0074 - acc: 0.7617 - val_loss: 0.7471 - val_f1_m: 0.0000e+00 - val_acc: 0.5905
Epoch 2/10
15/15 [==============================] - 2s 139ms/step - loss: 0.4863 - f1_m: 0.0574 - acc: 0.7716 - val_loss: 0.4655 - val_f1_m: 0.6299 - val_acc: 0.7714
Epoch 3/10
15/15 [==============================] - 2s 149ms/step - loss: 0.2251 - f1_m: 0.8333 - acc: 0.9202 - val_loss: 0.4776 - val_f1_m: 0.7690 - val_acc: 0.8190
Epoch 4/10
15/15 [==============================] - 2s 136ms/step - loss: 0.1018 - f1_m: 0.9175 - acc: 0.9596 - val_loss: 0.7191 - val_f1_m: 0.6874 - val_acc: 0.7905
Epoch 5/10
15/15 [==============================] - 2s 120ms/step - loss: 0.0479 - f1_m: 0.9709 - acc: 0.9847 - val_loss: 0.6118 - val_f1_m: 0.7792 - val_acc: 0.8190
Epoch 6/10
15/15 [==============================] - 2s 132ms/step - loss: 0.0295 - f1_m: 0.9851 - acc: 0.9923 - val_loss: 0.8886

<ipython-input-6-957c9328e0e5>:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred


Epoch 1/10
15/15 [==============================] - 12s 320ms/step - loss: 0.6026 - f1_m: 0.0267 - acc: 0.7366 - val_loss: 0.6560 - val_f1_m: 0.0000e+00 - val_acc: 0.5905
Epoch 2/10
15/15 [==============================] - 3s 199ms/step - loss: 0.4565 - f1_m: 0.3393 - acc: 0.8087 - val_loss: 0.4264 - val_f1_m: 0.7715 - val_acc: 0.8286
Epoch 3/10
15/15 [==============================] - 4s 276ms/step - loss: 0.1783 - f1_m: 0.8461 - acc: 0.9377 - val_loss: 0.7085 - val_f1_m: 0.6453 - val_acc: 0.7810
Epoch 4/10
15/15 [==============================] - 3s 181ms/step - loss: 0.0587 - f1_m: 0.9658 - acc: 0.9836 - val_loss: 0.9925 - val_f1_m: 0.5476 - val_acc: 0.7429
Epoch 5/10
15/15 [==============================] - 3s 189ms/step - loss: 0.0379 - f1_m: 0.9629 - acc: 0.9913 - val_loss: 0.4921 - val_f1_m: 0.8484 - val_acc: 0.8762
Epoch 6/10
15/15 [==============================] - 2s 165ms/step - loss: 0.0176 - f1_m: 0.9933 - acc: 0.9967 - val_loss: 1.3305 - val_f1_m: 0.6061 - val_acc: 0.7619

<ipython-input-6-957c9328e0e5>:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred
<ipython-input-6-957c9328e0e5>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test[list_kolom[0]] = df_test[list_kolom[0]].str.lower()


Epoch 1/10
15/15 [==============================] - 12s 286ms/step - loss: 0.5842 - f1_m: 0.0288 - acc: 0.7475 - val_loss: 0.6270 - val_f1_m: 0.0000e+00 - val_acc: 0.5905
Epoch 2/10
15/15 [==============================] - 3s 184ms/step - loss: 0.3246 - f1_m: 0.5689 - acc: 0.8579 - val_loss: 0.4449 - val_f1_m: 0.7750 - val_acc: 0.8286
Epoch 3/10
15/15 [==============================] - 3s 221ms/step - loss: 0.1191 - f1_m: 0.9001 - acc: 0.9585 - val_loss: 0.6403 - val_f1_m: 0.7679 - val_acc: 0.8286
Epoch 4/10
15/15 [==============================] - 2s 154ms/step - loss: 0.0284 - f1_m: 0.9825 - acc: 0.9913 - val_loss: 0.7983 - val_f1_m: 0.6703 - val_acc: 0.7714
Epoch 5/10
15/15 [==============================] - 3s 188ms/step - loss: 0.0152 - f1_m: 0.9940 - acc: 0.9967 - val_loss: 0.7732 - val_f1_m: 0.7471 - val_acc: 0.8095
Epoch 6/10
15/15 [==============================] - 2s 166ms/step - loss: 0.0026 - f1_m: 1.0000 - acc: 1.0000 - val_loss: 1.0671 - val_f1_m: 0.7097 - val_acc: 0.7905

...

<ipython-input-6-957c9328e0e5>:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred


Epoch 1/10
15/15 [==============================] - 11s 308ms/step - loss: 0.6056 - f1_m: 0.0061 - acc: 0.7574 - val_loss: 0.7367 - val_f1_m: 0.0000e+00 - val_acc: 0.5905
Epoch 2/10
15/15 [==============================] - 3s 198ms/step - loss: 0.4973 - f1_m: 0.2264 - acc: 0.7454 - val_loss: 0.6768 - val_f1_m: 0.0000e+00 - val_acc: 0.5905
Epoch 3/10
15/15 [==============================] - 2s 156ms/step - loss: 0.3107 - f1_m: 0.3951 - acc: 0.8328 - val_loss: 0.6669 - val_f1_m: 0.7097 - val_acc: 0.7905
Epoch 4/10
15/15 [==============================] - 2s 160ms/step - loss: 0.1146 - f1_m: 0.9243 - acc: 0.9607 - val_loss: 1.3020 - val_f1_m: 0.5911 - val_acc: 0.7524
Epoch 5/10
15/15 [==============================] - 2s 140ms/step - loss: 0.0682 - f1_m: 0.9717 - acc: 0.9858 - val_loss: 0.5578 - val_f1_m: 0.7547 - val_acc: 0.8095
Epoch 6/10
15/15 [==============================] - 2s 141ms/step - loss: 0.0351 - f1_m: 0.9846 - acc: 0.9923 - val_loss: 1.2365 - val_f1_m: 0.5404 - val_acc: 0.

<ipython-input-6-957c9328e0e5>:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred


Epoch 1/10
15/15 [==============================] - 11s 328ms/step - loss: 0.5966 - f1_m: 0.0205 - acc: 0.7301 - val_loss: 0.6407 - val_f1_m: 0.0000e+00 - val_acc: 0.5905
Epoch 2/10
15/15 [==============================] - 3s 178ms/step - loss: 0.2663 - f1_m: 0.7025 - acc: 0.8984 - val_loss: 0.3195 - val_f1_m: 0.8483 - val_acc: 0.8667
Epoch 3/10
15/15 [==============================] - 3s 176ms/step - loss: 0.1105 - f1_m: 0.9172 - acc: 0.9628 - val_loss: 0.4366 - val_f1_m: 0.8208 - val_acc: 0.8476
Epoch 4/10
15/15 [==============================] - 3s 170ms/step - loss: 0.0473 - f1_m: 0.9778 - acc: 0.9891 - val_loss: 0.7199 - val_f1_m: 0.7986 - val_acc: 0.8381
Epoch 5/10
15/15 [==============================] - 2s 157ms/step - loss: 0.0121 - f1_m: 0.9962 - acc: 0.9978 - val_loss: 0.9982 - val_f1_m: 0.7705 - val_acc: 0.8190
Epoch 6/10
15/15 [==============================] - 2s 160ms/step - loss: 0.0099 - f1_m: 0.9944 - acc: 0.9978 - val_loss: 0.9911 - val_f1_m: 0.7952 - val_acc: 0.8286

<ipython-input-6-957c9328e0e5>:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred


In [ ]:
for i in range(1):
  get_f1_each_representation('tsunami', i+1)